In [1]:
import copy
import numpy as np
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor
from fixedincomelib.utilities.risk_reporting import createValueReport

In [2]:
IndexManager.instance()

In [3]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [4]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [5]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]

In [6]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [7]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    strike=95.7,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

a = ve.value[1]
print("Baseline PV (a):", a)


RFR Future PV: ['USD', np.float64(3.2333518462344694e-07)]
Baseline PV (a): 3.2333518462344694e-07


In [8]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([99999999.99999993,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [9]:
yc.jacobian()

array([[-100.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,  -98.924743  ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,  -97.89682633,    0.        ,
           0.        ,    0.        , 

In [10]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, rfr_future, request="all", space="pv")

In [11]:
report['param_risk']

array([-1000000.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.])

In [12]:
report = createValueReport(vp, yc, rfr_future, request="all", space="quote")

In [13]:
report['quote_risk']

array([989247.43001761,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ])

V = df(0, T + 3M) (Fut - K)

dV/dFut = df 

dV/ddf = (Fut - K)

risk a disc * (Fut - K)

In [14]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B",to_date="2026-06-24")

In [15]:
yc.gradient_

array([-0.24489546, -0.2422043 , -0.24758662, -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        ])

In [16]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.71],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [17]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [18]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [19]:
yc2 = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [20]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    strike=95.7,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc2,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

b = ve.value[1]

RFR Future PV: ['USD', np.float64(-9892.721677124064)]


In [21]:
a

np.float64(3.2333518462344694e-07)

In [22]:
b

np.float64(-9892.721677124064)

In [23]:
(a-b)/0.01

np.float64(989272.1677447398)

In [24]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-09-24",
    maturityDate="2029-09-24",
    fixedFrequency="6M",
    floatFrequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.0369,
    notional=1000000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(1605.7752236218075)] Par Rate: 0.036468661366250155


In [25]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([-253183.68207042, -250401.44380591, -251324.4414439 ,
       -251324.4414439 , -244020.66154921, -241339.1158179 ,
       -242242.7587236 , -242242.7587236 , -235209.65570772,
       -235209.65570772, -233474.4512274 , -233474.4512274 ,
       -900948.72343599,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ])

In [26]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, ois_swap, request="all", space="quote")

In [27]:
report['param_risk']

array([ 2.62702201e+01,  2.62639413e+01, -2.02824604e+01, -2.04949755e+01,
        2.53321539e+01,  2.53020337e+01, -2.12993460e+01, -2.15040091e+01,
        2.43412800e+01,  2.45640163e+01, -2.26088065e+01, -2.28110297e+01,
        9.91213161e+05, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00])

In [28]:
report['quote_risk']

array([-2.59877478e+01, -2.57115650e+01,  1.96499967e+01,  1.96549676e+01,
       -2.40555257e+01, -2.37984582e+01,  1.98429617e+01,  1.98469807e+01,
       -2.22619120e+01, -2.22584594e+01,  2.03051456e+01,  2.03037565e+01,
        3.65683578e+06, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00])

In [29]:
swap = ProductOvernightSwap(
    effectiveDate="2026-09-24",
    maturityDate="2027-09-24",
    fixedFrequency="6M",
    floatFrequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.0369,
    notional=1000000,
    position="LONG"
)

In [30]:
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

a= ve.value[1]

OIS Swap PV: ['USD', np.float64(-1213.9488034361566)] Par Rate: 0.03818510951731962


In [31]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([    306.85928087,     303.48720086,     310.23136088,
           310.23136088, -242110.55788225, -239450.00230113,
       -240311.66490646, -240311.66490646,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ])

In [32]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, swap, request="all", space="pv")

In [33]:
report['param_risk']

array([  -3.06859281,   -3.06785938,   -3.16896239,   -3.20216608,
       2524.5780404 , 2521.57628937, 2554.94444223, 2579.49461949,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ])

In [34]:
report = createValueReport(vp, yc, swap, request="all", space="quote")
report['quote_risk']

array([    3.03559755,     3.00333697,     3.07014531,     3.07092196,
       -2397.3504991 , -2371.73140156, -2380.24513801, -2380.72723207,
           0.        ,     0.        ,     0.        ,     0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ])

In [35]:
MARKET_DF_3 = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.09],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [36]:
data_objs_3, dc_3 = build_yc_data_collection(MARKET_DF_3)

In [37]:
build_methods_3 = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [38]:
yc3 = YieldCurve("2025-09-24", dc_3, build_methods_3)
print("Curve components:", yc3.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [39]:
ve = ValuationEngineRegistry().new_valuation_engine(
    yc3, {"FUNDING INDEX": "SOFR-1B"}, swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(-1189.9746983791483)] Par Rate: 0.03815969853853918


In [40]:
b = ve.value[1]

In [41]:
a

np.float64(-1213.9488034361566)

In [42]:
b

np.float64(-1189.9746983791483)

In [43]:
(a-b)/0.01

np.float64(-2397.4105057008273)